In [1]:
import math 
import numpy as np
from sklearn import datasets
import pdb
import math
import numpy as np

# entropy bigger, the worse
def entropy(p1,n1):
    if(p1==0 and n1 ==0):
        return 1
    elif(p1==0 or n1 == 0):
        return 0
    else:
        pp = p1/(p1+n1)
        np = n1/(p1+n1)
        return -pp*math.log2(pp)-np*math.log2(np)

    
# gain bigger, the better classifier
# when stop, might be either 1. p,n ratio same, or 2. entropy returtn 1
def IG(p1,n1,p2,n2):
    num = p1+n1+p2+n2
    num1 = p1+n1
    num2 = p2+n2
    return entropy(p1+p2,n1+n2)-num1/num*entropy(p1,n1)-num2/num*entropy(p2,n2)

def acc(pred, true):
    #pdb.set_trace()
    diff = np.array(pred) - np.array(true)
    
    err = np.count_nonzero(diff)/len(diff)
    acc = 1 - err
    return acc

#print(IG(21,5,8,30))
#print(IG(18,33,11,2))






# build tree

In [2]:
def build_Tree(feature, target):
    node = dict()
    node['data'] = range(len(target))
    Tree = []
    Tree.append(node)
    t = 0


    # print(len(Tree))
    while(t<len(Tree)):
        idx = Tree[t]['data'] # what data(idx based) you have in the group
    #     print('idx: ', idx)

        if(sum(target[idx])==0): # every target is 0: is leaf, decision = 0 
            Tree[t]['leaf'] = 1
            Tree[t]['decision'] = 0
            #print('target idx == 0: ', Tree)

        elif(sum(target[idx])==len(idx)): # every target is 1: is leaf, decision = 1
            Tree[t]['leaf'] = 1
            Tree[t]['decision'] = 1
            #print('target idx == len idx: ', Tree)

        else:
            bestIG = 0
            for i in range(feature.shape[1]): # 4 features
                pool = list(set(feature[idx,i]))

                pool.sort()
                for j in range(len(pool)-1): # slice threshold: 1|23 or 12|3
                    thres = (pool[j]+pool[j+1])/2
                    G1 = []
                    G2 = []
                    for k in idx:
                        if(feature[k,i]<=thres):
                            G1.append(k)
                        else:
                            G2.append(k)
                    p1 = sum(target[G1]==1)
                    n1 = sum(target[G1]==0)
                    p2 = sum(target[G2]==1)
                    n2 = sum(target[G2]==0)
                    thisIG = IG(p1,n1,p2,n2)

                    if(thisIG>bestIG):
                        bestIG = thisIG
                        bestG1 = G1 # best left child
                        bestG2 = G2 # best right child
                        bestthres = thres # best threshold
                        bestf = i # best feature

            # append tree

            if(bestIG>0):
                Tree[t]['leaf'] = 0 # not leaf node when IG > 0 (when p1 p2 ratio same -> can't find better sol)
                Tree[t]['selectf'] = bestf
                Tree[t]['threshold'] = bestthres
                Tree[t]['child'] = [len(Tree),len(Tree)+1] # the idx of child

                node = dict()
                node['data'] = bestG1
                Tree.append(node)
                node = dict()
                node['data'] = bestG2
                Tree.append(node)
                print(Tree)

            else:
                Tree[t]['leaf'] = 1 # is leaf node
                if(sum(target[idx]==1)>sum(target[idx]==0)):
                    Tree[t]['decision'] = 1

                else:
                    Tree[t]['decision'] = 0
                #print(Tree)
        t+=1
        #print(len(Tree), t)

    return Tree


# test tree

In [3]:
def decision_Tree(feature, target, test_x, test_y):
    Tree = build_Tree(feature, target)

    pred = []
    true = []

    for i in range(len(test_y)):
        test_feature = test_x[i,:]
        now = 0 # where are you 
        while(Tree[now]['leaf']==0): # while not @ leaf
            bestf = Tree[now]['selectf']
            thres = Tree[now]['threshold']
            if(test_feature[bestf]<=thres):
                now = Tree[now]['child'][0]
            else:
                now = Tree[now]['child'][1]


        pred.append(Tree[now]['decision'])
        
        print(test_y[i],Tree[now]['decision'])

    
    return acc(pred, test_y)

# main function

In [4]:
#data = np.loadtxt('PlayTennis.txt',usecols=range(5),dtype=int) #load data
#feature = data[:,0:4]
#target = data[:,4]-1
# drop 2, remain (0,1)

iris = datasets.load_iris()
raw_feature = iris.data[:100]
raw_target = iris.target[:100]
# print(raw_data)
# print(raw_target)

total_acc = []
split_size = int(len(raw_feature)/10)
for i in range (10):
        range_idx = range(i * split_size, i * split_size + split_size)
        test_x = raw_feature[range_idx] 
        test_y = raw_target[range_idx] 
        feature = raw_feature[list(set(range(len(raw_feature))) - set(range_idx))] 
        target = raw_target[list(set(range(len(raw_feature))) - set(range_idx))] 
        # pdb.set_trace()
        total_acc.append(decision_Tree(feature, target, test_x, test_y))
    


[{'data': range(0, 90), 'leaf': 0, 'selectf': 2, 'threshold': 2.45, 'child': [1, 2]}, {'data': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]}, {'data': [40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89]}]
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
[{'data': range(0, 90), 'leaf': 0, 'selectf': 2, 'threshold': 2.45, 'child': [1, 2]}, {'data': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]}, {'data': [40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89]}]
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0


# answer of 10-fold cross validation total acc

In [5]:
total_acc


[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [15]:

# iris = datasets.load_iris()
# raw_feature = iris.data[:100]
# raw_target = iris.target[:100]
# # print(raw_data)
# data_idx = np.array(range(1,101))
# np.random.shuffle(data_idx)
# data_idx
# raw_feature = iris.data[:100]
# raw_target = iris.target[:100]

#raw_feature.sort(range(len(data_idx)), key = lambda k : data_idx[k])
#raw_feature

# s = [2,3,1,4,9]